In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os

In [2]:
import tensorflow
physical_devices = tensorflow.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import europarl

In [4]:
language_code = 'da'

In [5]:
mark_start = 'ssss '
mark_end = ' eeee'


In [6]:
europarl.maybe_download_and_extract(language_code = language_code)

Data has apparently already been downloaded and unpacked.


In [7]:
data_src = europarl.load_data(english=False,
                              language_code=language_code)
data_dest = europarl.load_data(english=True,
                               language_code=language_code,
                               start=mark_start,
                               end=mark_end)

In [8]:
data_src[2]

'Som De kan se, indfandt det store "år 2000-problem" sig ikke. Til gengæld har borgerne i en del af medlemslandene været ramt af meget forfærdelige naturkatastrofer.'

In [9]:
data_dest[2]

"ssss Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. eeee"

In [10]:
num_words = 10000

In [11]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [12]:
%%time
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

CPU times: user 2min, sys: 942 ms, total: 2min 1s
Wall time: 2min


In [13]:
%%time
tokenizer_dest = TokenizerWrap(texts=data_dest,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

CPU times: user 1min 28s, sys: 546 ms, total: 1min 28s
Wall time: 1min 28s


In [14]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(1968800, 47)
(1968800, 55)


In [15]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
print(token_start)
token_end = tokenizer_dest.word_index[mark_end.strip()]
print(token_end)

2
3


In [16]:
tokens_src[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3069,
       3374,   43,    7, 1386,  108, 1995,    7,  178,    9,    3,  302,
         19, 2076,    8,   20,   39,  285,  499,   69,  136,    5,  166,
         24,   10,   13], dtype=int32)

In [17]:
tokenizer_src.tokens_to_string(tokens_src[2])


'naturkatastrofer forfærdelige meget af ramt været medlemslandene af del en i borgerne har gengæld til ikke sig problem 2000 år store det se kan de som'

In [18]:
data_src[2]

'Som De kan se, indfandt det store "år 2000-problem" sig ikke. Til gengæld har borgerne i en del af medlemslandene været ramt af meget forfærdelige naturkatastrofer.'

In [19]:
encoder_input_data = tokens_src

In [20]:
decoder_input_data = tokens_dest[:, :-1]
decoder_input_data.shape

(1968800, 54)

In [21]:
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape

(1968800, 54)

In [22]:
decoder_input_data[2]

array([   2,  404,   19,   43,   26,   20,  618,    1, 1451,    5, 9785,
        174,    1,   81,    7,    9,  214,    4,   67, 2200,    9, 1596,
          4,  892, 1762,    8, 1480,  107, 5494,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [23]:
decoder_output_data[2]

array([ 404,   19,   43,   26,   20,  618,    1, 1451,    5, 9785,  174,
          1,   81,    7,    9,  214,    4,   67, 2200,    9, 1596,    4,
        892, 1762,    8, 1480,  107, 5494,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [24]:
encoder_input = Input(shape = (None,  ), name = 'encoder_input')

In [25]:
embedding_size = 128

In [26]:
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')

In [27]:
encoder_gru1 = GRU(512, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(512, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(512, name='encoder_gru3',
                   return_sequences=False)

In [28]:
def connect_encoder():
    net = encoder_input

    net = encoder_embedding(net)

    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    encoder_output = net
    
    return encoder_output

In [29]:
encoder_output = connect_encoder()

In [30]:
decoder_initial_state = Input(shape=(512,),
                              name='decoder_initial_state')

In [31]:
decoder_input = Input(shape=(None, ), name='decoder_input')

In [32]:
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [33]:

decoder_gru1 = GRU(512, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(512, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(512, name='decoder_gru3',
                   return_sequences=True)

In [34]:
decoder_dense = Dense(num_words, activation='linear', name = 'decoder_output')

In [35]:
def connect_decoder(initial_state):
    net = decoder_input

    net = decoder_embedding(net)

    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    decoder_output = decoder_dense(net)
    
    return decoder_output

In [36]:
decoder_output = connect_decoder(initial_state=encoder_output)

In [37]:
model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

In [38]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

In [39]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

In [40]:
model_train.compile(optimizer=RMSprop(lr=1e-3),
                    loss='sparse_categorical_crossentropy')

In [41]:
path_checkpoint = '21_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [42]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=3, verbose=1)

In [43]:
callback_tensorboard = TensorBoard(log_dir='./21_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [44]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard]

In [45]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

Error trying to load checkpoint.
Unable to open file (unable to open file: name = '21_checkpoint.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [46]:

x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [47]:
y_data = \
{
    'decoder_output': decoder_output_data
}


In [48]:
validation_split = 10000 / len(encoder_input_data)
validation_split

0.0050792360828931325

In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size = 50,
                epochs=10,
                validation_split=validation_split,
                callbacks=callbacks)

Epoch 1/10
20342/39176 [==============>...............] - ETA: 1:08:06 - loss: 9.0463